## 3.1 Churn Prediction Project
Dataset: https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

## 3.2 Data Preparation

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
!wget $data

--2025-10-09 08:08:51--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.003s  

2025-10-09 08:08:51 (24.5 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [3]:
df = pd.read_csv('course_lead_scoring.csv')

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
# If there are missing values:
# For categorical features, replace them with 'NA'
# For numerical features, replace with with 0.0

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [8]:
# for col in df.columns:
#     if df[col].dtype == 'object':  # categorical
#         df[col].fillna('NA', inplace=True)
#     else:  # numerical
#         df[col].fillna(0.0, inplace=True)


for col in df.columns:
    if df[col].dtype == 'object': # categorical
        df.fillna({col: 'NA'}, inplace=True)
    else:  # numerical
        df.fillna({col: 0.0}, inplace=True)

In [9]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [10]:
df.industry.mode()

0    retail
Name: industry, dtype: object

## 3.3 Correlation

- for numerical features

In [11]:
# Select only numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns
df_numerical = df[numerical_cols]

In [12]:
df_numerical.corrwith(df.converted)

number_of_courses_viewed    0.435914
annual_income               0.053131
interaction_count           0.374573
lead_score                  0.193673
converted                   1.000000
dtype: float64

## 3.4 Setting up the Validation framework

- performing the train/validation/test split with Scikit-learn

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# splitting dataset into 'train' and 'test' (80% - 20%)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [15]:
len(df_full_train), len(df_test)

(1169, 293)

In [16]:
# splitting the full training dataset into 'train' and 'valdation' (60% -20% -20%)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [17]:
len(df_train), len(df_val), len(df_test), len(df_full_train)

(876, 293, 293, 1169)

In [18]:
# defining 'y' of all 3 datasets
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [19]:
#  to remove the 'y' variables from the dataset i.e. the target variables
del df_train['converted']
del df_val['converted']
del df_test['converted']

## 3.5 EDA

- checking mssing values
- take a look at target variable (converted)
- checking numerical and categorical columns

In [20]:
# this gives the value/distribution of the target variable
df_full_train.converted.value_counts()

converted
1    710
0    459
Name: count, dtype: int64

In [21]:
# this gives the value/distribution of the target variable in percentage
# giving us the global conversion rate (positive class i.e. 'converted')
df_full_train.converted.value_counts(normalize=True)

converted
1    0.607357
0    0.392643
Name: proportion, dtype: float64

In [23]:
# the 'mean' can also be used to calculate this - global conversion rate
df_full_train.converted.mean()

np.float64(0.6073567151411463)

In [24]:
global_conversion_rate = df_full_train.converted.mean()
round(global_conversion_rate, 2)

np.float64(0.61)

In [25]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [26]:
categorical = ['location', 'lead_source', 'employment_status', 'industry']

## 3.6 Feature importance: Mutual information

- tells how much can be learned about one variable if the value of another is known.
- i.e. a measure of mutual dependence between two variables

In [27]:
from sklearn.metrics import mutual_info_score

In [ ]:
df_train

In [ ]:
y_train

In [30]:
# Calculating the mutual information score between y and other categorical 
# variables in the dataset. Using the training set only

In [31]:
mutual_info_score(y_train, df_train.lead_source)

0.03539624379726594

In [32]:
mutual_info_score(y_train, df_train.industry)

0.011574521435657112

In [33]:
mutual_info_score(y_train, df_train.employment_status)

0.012937677269442782

In [34]:
mutual_info_score(y_train, df_train.location)

0.004464157884038034

## 3.6 Training a Logistic Regression model

#### One-hot Encoding

- Using `scikit-learn` to encode categorical features

In [35]:
from sklearn.feature_extraction import DictVectorizer

In [36]:
df_train[['lead_source', 'industry', 'employment_status', 'location']].iloc[:10]

,lead_source,industry,employment_status,location
1077,paid_ads,retail,student,middle_east
463,organic_search,manufacturing,student,middle_east
842,paid_ads,technology,employed,north_america
835,NA,technology,employed,europe
837,organic_search,retail,student,australia
307,social_media,retail,student,south_america
269,social_media,finance,unemployed,africa
228,events,retail,student,middle_east
1234,organic_search,NA,employed,asia
321,organic_search,education,student,australia


In [80]:
# all categorical columns into dictionary format for easy vectorization
# dicts = df_train[['lead_source', 'industry', 'employment_status', 'location']].to_dict(orient='records')

# initializing the vectorizer
# sparse=False in order to display/return data as numpy array
# dv = DictVectorizer(sparse=False)

# training the vectorizer with the 'dicts' variable above
# dv.fit(dicts)

# dv.get_feature_names_out()

# to see what kind of data is stored
# dv.transform(dicts)

In [79]:
dv = DictVectorizer(sparse=False)

# for training dataset
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

# for validation dataset
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [ ]:
# Linear Regression in code
# def linear_regression(xi):
#     result = w0

#     for j in range(len(w)):
#         result = result + xi[j] * w[j]

#     return result

# note: w0, w and xi are not defined here yet.

In [ ]:
# Logistic Regression in code
# def logistic_regression(xi):
#     score = w0

#     for j in range(len(w)):
#         score = score + xi[j] * w[j]

#     result = sigmoid(score)
#     return result

####  Training logistic regression model with `scikit-learn`
- Train a model with scikit-learn
- apply to the validation dataset
- calculate the accuracy

In [81]:
from sklearn.linear_model import LogisticRegression

In [83]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [84]:
model.intercept_[0]

np.float64(-0.06914728027824993)

In [85]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [ ]:
# hard predictions
model.predict(X_train)

In [89]:
# soft predictions
model.predict_proba(X_train)

array([[0.42085658, 0.57914342],
       [0.12716509, 0.87283491],
       [0.41183895, 0.58816105],
       ...,
       [0.25265786, 0.74734214],
       [0.3302157 , 0.6697843 ],
       [0.14407824, 0.85592176]], shape=(876, 2))

In [97]:
y_pred = model.predict_proba(X_val)[:, 1]

In [98]:
conversion_decision = (y_pred >= 0.5)

In [99]:
(y_val == conversion_decision).mean()

np.float64(0.6996587030716723)

In [102]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = conversion_decision.astype(int)
df_pred['actual'] = y_val

In [104]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [109]:
df_pred.correct.mean().round(2)

np.float64(0.7)

In [105]:
df_pred

,probability,prediction,actual,correct
0,0.611922,1,0,False
1,0.799826,1,1,True
2,0.530213,1,0,False
3,0.471315,0,0,True
4,0.570661,1,0,False
...,...,...,...,...
288,0.419342,0,0,True
289,0.710539,1,1,True
290,0.418185,0,0,True
291,0.744835,1,1,True


In [100]:
conversion_decision.astype(int)

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0])

In [77]:
# dv = DictVectorizer(sparse=False)

# for test dataset
# test_dicts = df_test[categorical + numerical].to_dict(orient='records')
# X_test = dv.transform(test_dicts)